<a href="https://colab.research.google.com/github/Gabrielxdf/Ajuste-fino-de-um-modelo-SBERT/blob/main/SBERT_FINE_TUNNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install -q sentence_transformers

In [21]:
from torch.utils.data import DataLoader
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, InputExample, models, evaluation, losses, util
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

# Obtendo os dados

In [22]:
url = 'https://drive.google.com/file/d/1Wxc14dsJdUqOGFdauRs-Adw1Hy5Y3fEU/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

In [23]:
# Lendo e salvando para CSV
def obter_dados_csv():
    df_dados = pd.read_csv(url)
    df_dados.to_csv('dados.csv', index=False, encoding='utf-8')
    return df_dados

In [24]:
df_dados = obter_dados_csv()
df_dados

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,1
1,"Rafael Santos Avenida das Estrelas, 789 - Cida...",Desenvolvedor Full Stack Desenvolver soluções ...,3
2,"Amanda Costa Rua das Flores, 123 - Cidade das ...",Desenvolvedor Full Stack Desenvolver soluções ...,5
3,"Leonardo Santos Avenida das Estrelas, 789 - Ci...",Desenvolvedor Full Stack Desenvolver soluções ...,4
4,"Gabriela Lima Rua dos Programadores, 123 - Cid...",Desenvolvedor Full Stack Desenvolver soluções ...,5
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, 1...","Designer Gráfico Criar layouts, peças gráficas...",1
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",1
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",1
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",1


# Removendo os dígitos

In [25]:
df_dados["curriculos"] = df_dados["curriculos"].apply(lambda x: re.sub('\d+', '', x))
df_dados["vagas"] = df_dados["vagas"].apply(lambda x: re.sub('\d+', '', x))

In [26]:
min_max_scaler = MinMaxScaler()
min_max_scaler.set_output(transform='pandas');

In [27]:
df_dados["notas"] = min_max_scaler.fit_transform(df_dados["notas"].values.reshape(-1, 1))
# 1 -> 0.00
# 2 -> 0.25
# 3 -> 0.50
# 4 -> 0.75
# 5 -> 1.00
df_dados

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,0.00
1,"Rafael Santos Avenida das Estrelas, - Cidade ...",Desenvolvedor Full Stack Desenvolver soluções ...,0.50
2,"Amanda Costa Rua das Flores, - Cidade das Art...",Desenvolvedor Full Stack Desenvolver soluções ...,1.00
3,"Leonardo Santos Avenida das Estrelas, - Cidad...",Desenvolvedor Full Stack Desenvolver soluções ...,0.75
4,"Gabriela Lima Rua dos Programadores, - Cidade...",Desenvolvedor Full Stack Desenvolver soluções ...,1.00
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, ...","Designer Gráfico Criar layouts, peças gráficas...",0.00
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",0.00
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",0.00
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",0.00


# Preparando os dados para treinamento

In [28]:
data_examples = []
for index, row in df_dados.iterrows():
    data_examples.append(InputExample(texts=[row['curriculos'], row['vagas']], label=row['notas']))

# Dividindo os dados em 80% treino, 20% validação e 20% para testes
data_examples = shuffle(data_examples, random_state=42)
indice_treino = int(len(data_examples) * 0.6)
indice_val = int(len(data_examples) * 0.2)

train_examples = data_examples[:indice_treino]
val_examples = data_examples[indice_treino:indice_treino+indice_val]
test_examples = data_examples[indice_treino+indice_val:]

# Criando os Dataloaders do PyTorch
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)

# Treinamento

In [ ]:
checkpoint = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

word_embedding_model = models.Transformer(checkpoint, cache_dir=f'/content/drive/MyDrive/Colab Notebooks/model/{checkpoint}')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

embedding_before_ft = model.encode(test_examples[0].texts)

train_loss = losses.CosineSimilarityLoss(model)

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(val_examples, name='sbert')

model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, evaluator=evaluator, show_progress_bar=True, output_path=f'/content/drive/MyDrive/Colab Notebooks/model_FT/{checkpoint}', save_best_model=True)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/930 [00:00<?, ?it/s]

# Testando um sistema de recomendação de vagas para um currículo

In [ ]:
curriculos = list(set([test_example.texts[0] for test_example in test_examples]))
vagas = list(set([test_example.texts[1] for test_example in test_examples]))

In [ ]:
embedding_curriculos = model.encode(curriculos[0])
embedding_vagas = [model.encode(vaga) for vaga in vagas]
cosine_scores = util.cos_sim(embedding_curriculos, embedding_vagas)

In [ ]:
# Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)):
    for j in range(i, len(cosine_scores[0])):
        pairs.append({"index": [i, j], "score": cosine_scores[i][j]})

# Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x["score"], reverse=True)

In [ ]:
print(f' Currículo: {curriculos[i]} \n\n')
for pair in pairs[0:5]:
    i, j = pair["index"]
    print(f' Vaga: {vagas[j]} \n Similaridade predita após o ajuste fino: {pair["score"]} \n')